###  importing relevant libraries

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from collections import Counter
from tabulate import tabulate
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
import warnings
import sklearn
import time
import os
warnings.filterwarnings("ignore")

### Discovering Labels

In [2]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [3]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("Tthe folder could not be created!")

In [4]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [5]:
altime=0
#def most_frequent(List): 
#    return max(set(List), key = List.count) 


 
def most_frequent(List):
    occurence_count = Counter(List)
    occurence_count={k: v for k, v in sorted(occurence_count.items(), key=lambda item: item[1],reverse=True)}
    big=list(occurence_count.values())
    big=big.count(big[0])
    return list(occurence_count.keys())[np.random.randint(big)]


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
def create_exception(df): 
    exception_list=[]
    dominant_mac=[]
    for i in df['aggregated'].unique():
        k=df[df['aggregated']==i]
        for ii in ['MAC']:
            hist = {}
            for x in k[ii].values:
                hist[x] = hist.get(x, 0) + 1
            hist=dict(sorted(hist.items(), key=lambda item: item[1],reverse=True))
            temp=next(iter(hist))
            if temp not in dominant_mac:
                dominant_mac.append(temp)
            else:
                exception_list.append(temp)
    return exception_list




def merged(m_test,predict,step,mixed):
    second=time.time()
    mac_test=[]
    for q in m_test.index:
        mac_test.append(m_test[q])

    d_list=sorted(list(m_test.unique()))
    devices={}
    for q in d_list:
        devices[q]=[]    


    new_y=[0]*len(m_test)

    for q,qq in enumerate (mac_test):
        devices[qq].append(q)
    for q in devices:
        a = [devices[q][j:j + step] for j in range(0, len(devices[q]), step)]  
        for qq in a:
            step_list=[]
            for qqq in qq:
                step_list.append(predict[qqq])
            add=most_frequent(list(step_list))
            for qqq in qq:
                new_y[qqq]=add
    results=pd.DataFrame(m_test)
    results["aggregated"]=new_y
    results["normal"]=predict
    
    #MIXED METHOD
    if mixed:
        exception=create_exception(results)
        for q in exception:
            results.loc[results.MAC == q, 'aggregated'] = results['normal']

    return results["aggregated"].values,time.time()-second

## Calculation of evaluations

In [6]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):
    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [7]:
def ML(loop1,loop2,output_csv1,cols,step,mixed,dname):
    from sklearn.metrics import balanced_accuracy_score
    from sklearn.preprocessing import Normalizer
    
    for ii in ml_list:
        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        repetition=5
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):



            #TRAIN
            df = pd.read_csv(loop1,usecols=cols)
            m_train=df["MAC"]
            del df["MAC"]
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  

            #TEST
            df = pd.read_csv(loop2,usecols=cols)
            df = shuffle(df, random_state=42)
            m_test=df["MAC"]
            del df["MAC"]
            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes




            results_y=[]
            cv+=1
            results_y.append(y_test)


            #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time=(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time=(float((time.time()-second)) )
            if step==1:
                altime=0
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
            else:
                predict,altime=merged(m_test,predict,step,mixed)
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                
                
            lines.append(line)
            df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
            
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        #print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            graph_name=output_csv+"_confusion matrix.pdf"   
            plt.figure(figsize = (40,28))
            sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            cm.to_csv(graph_name.replace(".pdf",".csv"))
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            plt.clf()

            
            
            
            
            



# Machine learning applications 

# Aalto Dataset

In [8]:
feature= ['pck_size', 'Ether_type', 'LLC_ctrl', 'EAPOL_version', 'EAPOL_type', 'IP_ihl', 'IP_tos', 'IP_len', 'IP_flags', 'IP_DF', 'IP_ttl', 'IP_options', 'ICMP_code', 'TCP_dataofs', 'TCP_FIN', 'TCP_ACK', 'TCP_window', 'UDP_len', 'DHCP_options', 'BOOTP_hlen', 'BOOTP_flags', 'BOOTP_sname', 'BOOTP_file', 'BOOTP_options', 'DNS_qr', 'DNS_rd', 'DNS_qdcount', 'dport_class', 'payload_bytes', 'entropy',
"MAC",
'Label']


In [9]:
ml_list={"DT" :DecisionTreeClassifier()}



In [10]:
datasets=[['./csvs\\A211102.csv',
  './csvs\\A211105.csv',
  './csvs\\A211108.csv',
  './csvs\\A211109.csv',
  './csvs\\A211110.csv',
  './csvs\\A211112.csv',
  './csvs\\A211115.csv',
  './csvs\\A211116.csv',
  './csvs\\A211117.csv',
  './csvs\\A211118.csv',
  './csvs\\A211122.csv',
  './csvs\\A211123.csv',
  './csvs\\A211125.csv',
  './csvs\\A211206.csv',
  './csvs\\A211208.csv',
  './csvs\\A211223.csv',
  './csvs\\A211225.csv',
  './csvs\\A211228.csv',
  './csvs\\I211102.csv',
  './csvs\\I211103.csv',
  './csvs\\I211104.csv',
  './csvs\\I211108.csv',
  './csvs\\I211110.csv',
  './csvs\\I211112.csv',
  './csvs\\I211115.csv',
  './csvs\\I211118.csv',
  './csvs\\I211119.csv',
  './csvs\\I211122.csv'],
 ['./csvs\\A211103.csv', './csvs\\A211126.csv', './csvs\\A211207.csv'],
 ['./csvs\\A211124.csv',
  './csvs\\I211124.csv',
  './csvs\\I211125.csv',
  './csvs\\I211130.csv',
  './csvs\\I211201.csv',
  './csvs\\I211202.csv',
  './csvs\\I211203.csv',
  './csvs\\I211207.csv',
  './csvs\\I211208.csv',
  './csvs\\I211209.csv',
  './csvs\\I211210.csv',
  './csvs\\I211213.csv',
  './csvs\\I211215.csv',
  './csvs\\I211216.csv',
  './csvs\\I211220.csv',
  './csvs\\I211222.csv',
  './csvs\\I211223.csv'],
 ['./csvs\\I211116.csv', './csvs\\I211117.csv']]

In [11]:
datasets=[ ['./csvs\\I211116.csv', './csvs\\I211117.csv'],
 ['./csvs\\A211103.csv', './csvs\\A211126.csv', './csvs\\A211207.csv'],
 ['./csvs\\A211124.csv',
  './csvs\\I211124.csv',
  './csvs\\I211125.csv',
  './csvs\\I211130.csv',
  './csvs\\I211201.csv',
  './csvs\\I211202.csv',
  './csvs\\I211203.csv',
  './csvs\\I211207.csv',
  './csvs\\I211208.csv',
  './csvs\\I211209.csv',
  './csvs\\I211210.csv',
  './csvs\\I211213.csv',
  './csvs\\I211215.csv',
  './csvs\\I211216.csv',
  './csvs\\I211220.csv',
  './csvs\\I211222.csv',
  './csvs\\I211223.csv']]

In [12]:
say=0
dataset="./results/vs/"
folder(dataset)
for no, i in enumerate(datasets):
    for ii in i:
        for iii in tqdm(i):
            say+=1
            train=ii
            test=iii
            mixed=False
            step=1
            sayac=1
            dataset="./results/vs/"
            output_csv=f"{dataset}/@{ii[7:-4]}@{iii[7:-4]}.csv"
            dataset=f"{ii[7:-4]}-{iii[7:-4]}"
            target_names=target_name(test)
            ML(train,test,output_csv,feature,step,mixed,dataset+"_"+str(step))   


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  I211116-I211116_1    0     1  DT    0.952    0.949    0.95  0.949  0.948  0.949      0    0.583     0.173          0
 1  I211116-I211116_1    1     2  DT    0.952    0.949    0.95  0.949  0.948  0.949      0    1.644     0.156          0
 2  I211116-I211116_1    2     3  DT    0.952    0.949    0.95  0.949  0.948  0.949      0    1.561     0.152          0
 3  I211116-I211116_1    3     4  DT    0.952    0.949    0.95  0.949  0.948  0.949      0    1.701     0.201          0
 4  I211116-I211116_1    4     5  DT    0.952    0.949    0.95  0.949  0.948  0.949      0    1.669     0.188          0



 50%|██████████████████████████████████████████                                          | 1/2 [00:38<00:38, 38.32s/it]

    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  I211116-I211117_1    0     1  DT    0.789     0.78   0.819   0.78  0.742  0.779      0    1.6       0.19           0
 1  I211116-I211117_1    1     2  DT    0.789     0.78   0.82    0.78  0.744  0.779      0    1.614     0.201          0
 2  I211116-I211117_1    2     3  DT    0.789     0.78   0.838   0.78  0.755  0.779      0    1.685     0.204          0
 3  I211116-I211117_1    3     4  DT    0.789     0.78   0.826   0.78  0.748  0.779      0    1.482     0.194          0
 4  I211116-I211117_1    4     5  DT    0.789     0.78   0.836   0.78  0.754  0.779      0    1.445     0.22           0



  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

...

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

In [13]:
datasets=[['./csvs\\A211102.csv',
  './csvs\\A211105.csv',
  './csvs\\A211108.csv',
  './csvs\\A211109.csv',
  './csvs\\A211110.csv',
  './csvs\\A211112.csv',
  './csvs\\A211115.csv',
  './csvs\\A211116.csv',
  './csvs\\A211117.csv',
  './csvs\\A211118.csv',
  './csvs\\A211122.csv',
  './csvs\\A211123.csv',
  './csvs\\A211125.csv',
  './csvs\\A211206.csv',
  './csvs\\A211208.csv',
  './csvs\\A211223.csv',
  './csvs\\A211225.csv',
  './csvs\\A211228.csv',
  './csvs\\I211102.csv',
  './csvs\\I211103.csv',
  './csvs\\I211104.csv',
  './csvs\\I211108.csv',
  './csvs\\I211110.csv',
  './csvs\\I211112.csv',
  './csvs\\I211115.csv',
  './csvs\\I211118.csv',
  './csvs\\I211119.csv',
  './csvs\\I211122.csv']]

In [14]:
say=0
dataset="./results/vs/"
folder(dataset)
for no, i in enumerate(datasets):
    for ii in i:
        for iii in tqdm(i):
            say+=1
            train=ii
            test=iii
            mixed=False
            step=1
            sayac=1
            dataset="./results/vs/"
            output_csv=f"{dataset}/@{ii[7:-4]}@{iii[7:-4]}.csv"
            dataset=f"{ii[7:-4]}-{iii[7:-4]}"
            target_names=target_name(test)
            ML(train,test,output_csv,feature,step,mixed,dataset+"_"+str(step))   


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  A211102-A211102_1    0     1  DT    0.952    0.928   0.948  0.928  0.935   0.95      0    1.655     0.147          0
 1  A211102-A211102_1    1     2  DT    0.952    0.928   0.948  0.928  0.935   0.95      0    1.252     0.143          0
 2  A211102-A211102_1    2     3  DT    0.952    0.928   0.948  0.928  0.935   0.95      0    1.387     0.149          0
 3  A211102-A211102_1    3     4  DT    0.952    0.928   0.948  0.928  0.935   0.95      0    1.413     0.155          0
 4  A211102-A211102_1    4     5  DT    0.952    0.928   0.948  0.928  0.935   0.95      0    1.568     0.206          0



  4%|██▉                                                                                | 1/28 [00:40<18:11, 40.44s/it]

    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  A211102-A211105_1    0     1  DT    0.915    0.845   0.888  0.845  0.851   0.91      0    1.29      0.14           0
 1  A211102-A211105_1    1     2  DT    0.915    0.845   0.888  0.845  0.851   0.91      0    1.265     0.178          0
 2  A211102-A211105_1    2     3  DT    0.915    0.845   0.888  0.845  0.851   0.91      0    1.319     0.15           0
 3  A211102-A211105_1    3     4  DT    0.915    0.845   0.888  0.845  0.851   0.91      0    1.301     0.156          0
 4  A211102-A211105_1    4     5  DT    0.915    0.845   0.888  0.845  0.851   0.91      0    1.362     0.146          0



  7%|█████▉                                                                             | 2/28 [01:15<16:14, 37.49s/it]

...

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

<Figure size 4000x2800 with 0 Axes>

In [ ]:
['./csvs\\A211102.csv',
  './csvs\\A211105.csv',
  './csvs\\A211108.csv',
  './csvs\\A211109.csv',
  './csvs\\A211110.csv',
  './csvs\\A211112.csv',
  './csvs\\A211115.csv',
  './csvs\\A211116.csv',
  './csvs\\A211117.csv',
  './csvs\\A211118.csv',
  './csvs\\A211122.csv',
  './csvs\\A211123.csv',
  './csvs\\A211125.csv',
  './csvs\\A211206.csv',
  './csvs\\A211208.csv',
  './csvs\\A211223.csv',
  './csvs\\A211225.csv',
  './csvs\\A211228.csv',
  './csvs\\I211102.csv',
  './csvs\\I211103.csv',
  './csvs\\I211104.csv',
  './csvs\\I211108.csv',
  './csvs\\I211110.csv',
  './csvs\\I211112.csv',
  './csvs\\I211115.csv',
  './csvs\\I211118.csv',
  './csvs\\I211119.csv',
  './csvs\\I211122.csv']

In [ ]:
say=0
dataset="./results/vs/"
folder(dataset)
for no, ii in enumerate(  ['./csvs\\I211104.csv',  './csvs\\I211108.csv',  './csvs\\I211110.csv',  './csvs\\I211112.csv',  './csvs\\I211115.csv',  './csvs\\I211118.csv',  './csvs\\I211119.csv',  './csvs\\I211122.csv']):
        for iii in tqdm(['./csvs\\A211102.csv',  './csvs\\A211105.csv',  './csvs\\A211108.csv',  './csvs\\A211109.csv',  './csvs\\A211110.csv',  './csvs\\A211112.csv',  './csvs\\A211115.csv',  './csvs\\A211116.csv',  './csvs\\A211117.csv',  './csvs\\A211118.csv',  './csvs\\A211122.csv',  './csvs\\A211123.csv',  './csvs\\A211125.csv',  './csvs\\A211206.csv',  './csvs\\A211208.csv',  './csvs\\A211223.csv',  './csvs\\A211225.csv',  './csvs\\A211228.csv',  './csvs\\I211102.csv',  './csvs\\I211103.csv',  './csvs\\I211104.csv',  './csvs\\I211108.csv',  './csvs\\I211110.csv',  './csvs\\I211112.csv',  './csvs\\I211115.csv',  './csvs\\I211118.csv',  './csvs\\I211119.csv',  './csvs\\I211122.csv']):
            say+=1
            train=ii
            test=iii
            mixed=False
            step=1
            sayac=1
            dataset="./results/vs/"
            output_csv=f"{dataset}/@{ii[7:-4]}@{iii[7:-4]}.csv"
            dataset=f"{ii[7:-4]}-{iii[7:-4]}"
            target_names=target_name(test)
            ML(train,test,output_csv,feature,step,mixed,dataset+"_"+str(step))   


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  I211104-A211102_1    0     1  DT    0.849    0.829   0.853  0.829  0.816  0.841      0    1.49      0.157          0
 1  I211104-A211102_1    1     2  DT    0.849    0.829   0.853  0.829  0.816  0.841      0    1.529     0.141          0
 2  I211104-A211102_1    2     3  DT    0.848    0.829   0.852  0.829  0.816  0.841      0    1.434     0.134          0
 3  I211104-A211102_1    3     4  DT    0.849    0.829   0.852  0.829  0.816  0.841      0    1.365     0.135          0
 4  I211104-A211102_1    4     5  DT    0.849    0.829   0.852  0.829  0.816  0.841      0    1.516     0.151          0



  4%|██▉                                                                                | 1/28 [00:36<16:12, 36.03s/it]

    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  I211104-A211105_1    0     1  DT    0.872    0.876   0.88   0.876  0.849  0.866      0    1.399     0.157          0
 1  I211104-A211105_1    1     2  DT    0.872    0.876   0.88   0.876  0.849  0.866      0    1.468     0.135          0
 2  I211104-A211105_1    2     3  DT    0.872    0.876   0.88   0.876  0.849  0.866      0    1.5       0.151          0
 3  I211104-A211105_1    3     4  DT    0.872    0.876   0.88   0.876  0.849  0.866      0    1.451     0.146          0
 4  I211104-A211105_1    4     5  DT    0.872    0.876   0.879  0.876  0.849  0.866      0    1.504     0.153          0



  7%|█████▉                                                                             | 2/28 [01:11<15:35, 35.96s/it]

...

 86%|██████████████████████████████████████████████████████████████████████▎           | 24/28 [15:37<02:39, 39.98s/it]

    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  I211122-I211115_1    0     1  DT    0.793    0.787   0.81   0.787  0.775  0.783      0    1.529     0.182          0
 1  I211122-I211115_1    1     2  DT    0.793    0.786   0.81   0.786  0.775  0.783      0    1.575     0.195          0
 2  I211122-I211115_1    2     3  DT    0.793    0.786   0.81   0.786  0.775  0.783      0    1.59      0.19           0
 3  I211122-I211115_1    3     4  DT    0.793    0.786   0.808  0.786  0.775  0.782      0    1.57      0.195          0
 4  I211122-I211115_1    4     5  DT    0.793    0.786   0.809  0.786  0.775  0.783      0    1.5       0.205          0



 89%|█████████████████████████████████████████████████████████████████████████▏        | 25/28 [16:17<01:59, 39.97s/it]

    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  I211122-I211118_1    0     1  DT    0.895    0.845   0.883  0.845  0.845  0.889      0    1.688     0.175          0
 1  I211122-I211118_1    1     2  DT    0.895    0.845   0.883  0.845  0.845  0.889      0    1.388     0.172          0
 2  I211122-I211118_1    2     3  DT    0.895    0.845   0.883  0.845  0.845  0.889      0    1.453     0.153          0
 3  I211122-I211118_1    3     4  DT    0.895    0.845   0.879  0.845  0.843  0.889      0    1.405     0.18           0
 4  I211122-I211118_1    4     5  DT    0.895    0.845   0.867  0.845  0.837  0.889      0    1.455     0.165          0



 93%|████████████████████████████████████████████████████████████████████████████▏     | 26/28 [16:57<01:19, 39.97s/it]

In [ ]:
!shutdown /s /t 360